In [ ]:
# default_exp vulnerabilidad

In [ ]:
%load_ext autoreload
%autoreload 2

# Vulnerabilidad

> Métodos para dignosticar la vulnerabilidad a COVID-19 de entidades y municipios basandonos en sus 
indicadores sociales, económicos, de salud y de infraestructura

In [ ]:
# export
from ipywidgets import IntProgress
from IPython.display import display
import time

from covid19_vulnerabilidad_mex.datos import *

import pandas as pd
import geopandas as gpd
import mapclassify

import seaborn as sns
import glob
import os
import matplotlib.pyplot as plt
import numpy as np

from datetime import timedelta
import datetime

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.cross_decomposition import PLSRegression

## Exploración visual

### show_feature_importances

In [ ]:
# exports

def show_feature_importances(rf):
    importances = rf.feature_importances_

    std = np.std([tree.feature_importances_ for tree in rf.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

    # Plot the feature importances of the forest
    f, ax = plt.subplots(figsize=(15, 10))
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), X.columns[indices])
    plt.xlim([-1, X.shape[1]])
    plt.show()

In [ ]:
# exports

def mostrar_coeficientes_PLS(pls):
    f, ax = plt.subplots(figsize=(15, 10))
    sns.barplot(y='nombre', x='coef', data=feats_df, color='c')

In [ ]:
# exports

def agregar_conteo_pruebas(covid_municipal, solo_covid=True):
    cols_localidad = ['ENTIDAD_RES', 'MUNICIPIO_RES', 'CLAVE_MUNICIPIO_RES']
    count_tested = covid_municipal[cols_localidad + ['conteo']].groupby(cols_localidad).sum()
    count_tested.reset_index(inplace=True)
    count_tested.rename(columns={'conteo': 'total_pruebas'}, inplace=True)

    covid_municipal = covid_municipal.merge(count_tested, on=cols_localidad, how='left')


    covid_municipal['casos_frac'] = 100 * covid_municipal['conteo'] / covid_municipal['total_pruebas']
    covid_municipal['tasa_covid_letal'] = 100 * covid_municipal['defunciones'] / covid_municipal['conteo']

    if solo_covid:
        covid_municipal = covid_municipal.query('RESULTADO == "Positivo SARS-CoV-2"')
    return covid_municipal

In [ ]:
# exports

def agregar_tasas_municipales(casos_mun_df):
    casos_mun_df['covid_confirmados_100k'] = 100000 * casos_mun_df['conteo'] / casos_mun_df['pob2020']
    casos_mun_df['covid_defun_100k'] = 100000 * casos_mun_df['defunciones'] / casos_mun_df['pob2020']
    casos_mun_df['tasa_covid_letal'] = 100 * casos_mun_df['defunciones'] / casos_mun_df['conteo']

    # covid_municipal = covid_municipal.query('RESULTADO == "Positivo SARS-CoV-2"').copy()

    # casos_mun_df = gpd.GeoDataFrame(casos_mun_df, geometry='geometry')
    
    return casos_mun_df

In [ ]:
# exports

def caracteristicas_modelos_municipios(mun_df, poblaciones=False, i_vuln=False):
    pob_vars = []
    if not poblaciones:
        pob_vars = list(mun_df.columns[mun_df.columns.str.contains('_pob')])
        pob_vars = pob_vars + ['mayores_65', 'pob2020', 'pt_2015', 'pob_menore']
        pob_vars = pob_vars + ['sin_dere_1', 'sin_derech', 'carencias_']
    i_vuln_vars = []
    if not i_vuln:
        i_vuln_vars = ['i_vuln_salud', 'i_vuln_cobertura', 'i_vuln_econo', 'i_vuln_social',
                       'i_vuln_gen', 'i_vuln_infraestructura']
        
    columnas_numericas = mun_df.select_dtypes(include=np.number).columns
    
    otras_vars = ['covid_defun_100k', 'tasa_covid_letal', 'defunciones',
                  'INTUBADO_BIN', 'UCI_BIN', 'UCIs', 'tasa_uci', 'total_pruebas',
                  'area_cart', 'area', 'densi', 'casos_frac', 'area_km2', 'conteo',
                  'oid', 'covid_confrimados_100k', 'id', 'vulnerabilidad_ambiental_num']
    
    caracteristicas = set(columnas_numericas).difference(pob_vars + i_vuln_vars + otras_vars)
 
    return caracteristicas
    

In [ ]:
fecha = '200601'
covid_municipal = tabla_covid_indicadores_municipales(fecha)

In [ ]:
covid_municipal = agregar_tasas_municipales(covid_municipal)

In [ ]:
covid_municipal

,CLAVE_MUNICIPIO_RES,CLAVE_ENTIDAD_RES,MUNICIPIO_RES,geometry,RESULTADO,ENTIDAD_RES,conteo,defunciones,pt_2015,an_2015,...,total_hospitales_publicos,total_camas_publicos,hospitales_privados,total_de_camas_privados,total_camas,pob2020,CVE_MUN,covid_confrimados_100k,covid_defun_100k,tasa_covid_letal
0,01001,01,AGUASCALIENTES,"POLYGON ((-102.09775 22.02325, -102.11598 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,652.0,30.0,877190,2.0570,...,8,1228,17,43,1271,961977,001,67.777088,3.118578,4.601227
4,01002,01,ASIENTOS,"POLYGON ((-102.04348 22.29392, -102.05861 22.3...",Positivo SARS-CoV-2,AGUASCALIENTES,8.0,0.0,46464,4.4207,...,0,0,0,0,0,50864,002,15.728216,0.000000,0.000000
6,01003,01,CALVILLO,"POLYGON ((-102.57625 21.96778, -102.59749 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,11.0,0.0,56048,4.7805,...,1,46,1,0,46,60760,003,18.104016,0.000000,0.000000
11,01004,01,COSÍO,"POLYGON ((-102.26605 22.40372, -102.28993 22.4...",Positivo SARS-CoV-2,AGUASCALIENTES,15.0,0.0,15577,4.3233,...,0,0,0,0,0,16918,004,88.662963,0.000000,0.000000
13,01005,01,JESÚS MARÍA,"POLYGON ((-102.31034 22.03716, -102.33259 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,30.0,2.0,120405,3.2445,...,1,297,0,0,297,130184,005,23.044307,1.536287,6.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,32054,32,VILLA HIDALGO,"POLYGON ((-101.65599 22.51381, -101.67475 22.5...",Positivo SARS-CoV-2,ZACATECAS,1.0,1.0,19155,8.1332,...,0,0,0,0,0,20177,054,4.956138,4.956138,100.000000
7385,32055,32,VILLANUEVA,"POLYGON ((-102.69428 22.62230, -102.70306 22.6...",Positivo SARS-CoV-2,ZACATECAS,2.0,0.0,30240,4.7985,...,1,22,1,0,22,31804,055,6.288517,0.000000,0.000000
7386,32056,32,ZACATECAS,"POLYGON ((-102.58542 22.81149, -102.59311 22.7...",Positivo SARS-CoV-2,ZACATECAS,60.0,4.0,146147,1.2432,...,3,337,5,0,337,155533,056,38.577022,2.571801,6.666667
7390,32057,32,TRANCOSO,"POLYGON ((-102.24610 22.73451, -102.24578 22.7...",Positivo SARS-CoV-2,ZACATECAS,3.0,0.0,19413,5.6122,...,1,12,0,0,12,20285,057,14.789253,0.000000,0.000000


In [ ]:
caracteristicas_mun = caracteristicas_modelos_municipios(covid_municipal)
caracteristicas_mun

{'an_2015',
 'bi_2015',
 'carencias',
 'carencias3',
 'consulto_1',
 'consultori',
 'farmacias',
 'hospitales_imss',
 'hospitales_issste',
 'hospitales_pemex',
 'hospitales_privados',
 'hospitales_sedena',
 'hospitales_semar',
 'hospitales_sme',
 'hospitales_ssa',
 'ic_ali',
 'ic_asalud',
 'ic_cv',
 'ic_rezedu',
 'ic_sbv',
 'ic_segsoc',
 'irs_2015',
 'lmex_2015',
 'ne614_015',
 'npnv',
 'plb',
 'plbm',
 'pob_total',
 'poblacion',
 'pobreza',
 'pobreza_e',
 'pobreza_m',
 'porc_carencia_salud',
 'sins_15',
 'tasa_cancer',
 'tasa_cardiacas',
 'tasa_diabetes',
 'tasa_pulmonares',
 'total_camas',
 'total_camas_publicos',
 'total_de_camas_privados',
 'total_hospitales_publicos',
 'vna_2015',
 'vnd_2015',
 'vne_2015',
 'vnl_2015',
 'vnr_2015',
 'vns_2015',
 'vpt_2015',
 'vul_car',
 'vul_ing'}

In [ ]:
#exports

def calificar_municipios_letalidad(mun_df, regr, caracteristicas, etiqueta):
    not_na_row = mun_df[caracteristicas].notnull().all(axis=1)
    X = mun_df.loc[not_na_row, caracteristicas]
    Y_pred = regr.predict(X)
    mun_df.loc[not_na_row, etiqueta] = Y_pred
    
    return mun_df
    

In [ ]:
#exports

def calificar_municipios_letalidad_formato_largo(mun_df, regr, caracteristicas, modelo, dia_ajuste):
    mun_df = mun_df.copy()
    not_na_row = mun_df[caracteristicas].notnull().all(axis=1)
    X = mun_df.loc[not_na_row, caracteristicas]
    Y_pred = regr.predict(X)
    mun_df['modelo'] = modelo
    mun_df['dia_ajuste'] = dia_ajuste
    mun_df.loc[not_na_row, 'valor'] = Y_pred
    
    return mun_df
    

In [ ]:
#exports

def ajustar_pls_letalidad(municipios_df, caracteristicas, min_casos=20):
    data_train = municipios_df.loc[municipios_df[caracteristicas].notna().all(axis=1)]

    X = data_train.query(f'conteo > {min_casos}')[caracteristicas]
    Y = data_train.query(f'conteo > {min_casos}')['tasa_covid_letal']

    # X['i_vuln_econo'] = -X['i_vuln_econo']

    pls2 = PLSRegression(n_components=1)
    pls2.fit(X, Y)
    pls2.coef_ = pls2.coef_.flatten()
    
    return pls2

In [ ]:
pls = ajustar_pls_letalidad(municipios_df=covid_municipal, caracteristicas=caracteristicas_mun)

In [ ]:
#exports

def seleccionar_caracteristicas(regr, X, caracteristicas):
    sel = SelectFromModel(regr, prefit=True)

    caracteristicas_selec = list(X[caracteristicas].columns[sel.get_support()])
    return caracteristicas_selec

In [ ]:
seleccionar_caracteristicas(pls, covid_municipal, caracteristicas_mun)

['vpt_2015',
 'pobreza',
 'ic_rezedu',
 'vnl_2015',
 'vna_2015',
 'pobreza_e',
 'vul_ing',
 'pobreza_m',
 'lmex_2015',
 'vnr_2015',
 'ic_cv',
 'an_2015',
 'irs_2015',
 'ic_segsoc',
 'carencias3',
 'hospitales_pemex',
 'carencias',
 'ic_sbv',
 'ic_ali',
 'plbm',
 'bi_2015',
 'vnd_2015',
 'plb',
 'npnv']

In [ ]:
# exports

def calcular_indices_vulnerabilidad(fecha_0, fecha_1, vulnerabilidad='fecha_0', periodo=True):
    covid_municipal_0 = tabla_covid_indicadores_municipales(fecha_0)
    covid_municipal_0 = agregar_tasas_municipales(covid_municipal_0)
    
    covid_municipal_1 = tabla_covid_indicadores_municipales(fecha_1)
    covid_municipal_1 = agregar_tasas_municipales(covid_municipal_1)
    
    caracteristicas = caracteristicas_modelos_municipios(covid_municipal_0)    
    
    resultados = covid_municipal_1.copy()
    
    # rf = ajustar_rf_municipios(covid_municipal_0, fecha_0, caracteristicas)
    # caracteristicas_rf = seleccionar_caracteristicas(rf, covid_municipal_0, caracteristicas)
    # resultados = calificar_municipios_letalidad(resultados, rf, caracteristicas, etiqueta='i_RF_0')
    
    pls = ajustar_pls_letalidad(covid_municipal_0, caracteristicas)
    resultados = calificar_municipios_letalidad(resultados, pls, caracteristicas, etiqueta='i_PLS_0')
    
    pls = ajustar_pls_letalidad(covid_municipal_1, caracteristicas)
    resultados = calificar_municipios_letalidad(resultados, pls, caracteristicas, etiqueta='i_PLS_1')
    
    # pls = ajustar_pls_letalidad(covid_municipal_0, caracteristicas_rf)
    # resultados = calificar_municipios_letalidad(resultados, pls, caracteristicas, etiqueta='i_PLS_RF_1')
    
    return resultados


In [ ]:
evaluacion = calcular_indices_vulnerabilidad('200501', '200603')

In [ ]:
evaluacion

,CLAVE_MUNICIPIO_RES,CLAVE_ENTIDAD_RES,MUNICIPIO_RES,geometry,RESULTADO,ENTIDAD_RES,conteo,defunciones,pt_2015,an_2015,...,hospitales_privados,total_de_camas_privados,total_camas,pob2020,CVE_MUN,covid_confrimados_100k,covid_defun_100k,tasa_covid_letal,i_PLS_0,i_PLS_1
0,01001,01,AGUASCALIENTES,"POLYGON ((-102.09775 22.02325, -102.11598 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,731.0,31.0,877190,2.0570,...,17,43,1271,961977,001,75.989343,3.222530,4.240766,7.114357,8.872513
4,01002,01,ASIENTOS,"POLYGON ((-102.04348 22.29392, -102.05861 22.3...",Positivo SARS-CoV-2,AGUASCALIENTES,9.0,0.0,46464,4.4207,...,0,0,0,50864,002,17.694243,0.000000,0.000000,14.503379,12.282665
6,01003,01,CALVILLO,"POLYGON ((-102.57625 21.96778, -102.59749 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,11.0,0.0,56048,4.7805,...,1,0,46,60760,003,18.104016,0.000000,0.000000,13.304755,12.091215
11,01004,01,COSÍO,"POLYGON ((-102.26605 22.40372, -102.28993 22.4...",Positivo SARS-CoV-2,AGUASCALIENTES,16.0,0.0,15577,4.3233,...,0,0,0,16918,004,94.573827,0.000000,0.000000,11.999298,11.296056
13,01005,01,JESÚS MARÍA,"POLYGON ((-102.31034 22.03716, -102.33259 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,33.0,2.0,120405,3.2445,...,0,0,297,130184,005,25.348737,1.536287,6.060606,9.987346,10.395490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,32054,32,VILLA HIDALGO,"POLYGON ((-101.65599 22.51381, -101.67475 22.5...",Positivo SARS-CoV-2,ZACATECAS,1.0,1.0,19155,8.1332,...,0,0,0,20177,054,4.956138,4.956138,100.000000,17.867164,13.647780
7385,32055,32,VILLANUEVA,"POLYGON ((-102.69428 22.62230, -102.70306 22.6...",Positivo SARS-CoV-2,ZACATECAS,3.0,0.0,30240,4.7985,...,1,0,22,31804,055,9.432776,0.000000,0.000000,14.235058,12.322690
7386,32056,32,ZACATECAS,"POLYGON ((-102.58542 22.81149, -102.59311 22.7...",Positivo SARS-CoV-2,ZACATECAS,64.0,4.0,146147,1.2432,...,5,0,337,155533,056,41.148824,2.571801,6.250000,7.461657,9.421965
7390,32057,32,TRANCOSO,"POLYGON ((-102.24610 22.73451, -102.24578 22.7...",Positivo SARS-CoV-2,ZACATECAS,3.0,0.0,19413,5.6122,...,0,0,12,20285,057,14.789253,0.000000,0.000000,15.426984,12.743246


In [ ]:
evaluacion[evaluacion.i_PLS_0.notna()].plot(column='i_PLS_0', cmap='RdYlBu_r', scheme="Quantiles",
         figsize=(15, 20), legend=True, k=3)

In [ ]:
evaluacion[evaluacion.tasa_covid_letal.notna()].plot(column='tasa_covid_letal', cmap='RdYlBu_r', scheme="EqualInterval",
         figsize=(15, 20), legend=True, k=10)

In [ ]:
evaluacion[evaluacion.i_PLS_1.notna()].plot(column='i_PLS_1', cmap='RdYlBu_r', scheme="Quantiles",
         figsize=(15, 20), legend=True, k=3)

In [ ]:
# exports

def calcular_indices_vulnerabilidad_formato_largo(fecha_0, fecha_1):
    covid_municipal_0 = tabla_covid_indicadores_municipales(fecha_0)
    covid_municipal_0 = agregar_tasas_municipales(covid_municipal_0)
    
    covid_municipal_1 = tabla_covid_indicadores_municipales(fecha_1)
    covid_municipal_1 = agregar_tasas_municipales(covid_municipal_1)
    
    caracteristicas = caracteristicas_modelos_municipios(covid_municipal_0)    
    
    # rf = ajustar_rf_municipios(covid_municipal_0, fecha_0, caracteristicas)
    # caracteristicas_rf = seleccionar_caracteristicas(rf, covid_municipal_0, caracteristicas)
    # resultados = calificar_municipios_letalidad(resultados, rf, caracteristicas, etiqueta='i_RF_0')
    
    pls = ajustar_pls_letalidad(covid_municipal_0, caracteristicas)
    resultados = calificar_municipios_letalidad_formato_largo(covid_municipal_0, pls, caracteristicas,
                                                modelo='PLS', dia_ajuste=fecha_0)
    
    pls = ajustar_pls_letalidad(covid_municipal_1, caracteristicas)
    resultados_temp = calificar_municipios_letalidad_formato_largo(covid_municipal_1, pls, caracteristicas,
                                                     modelo='PLS', dia_ajuste=fecha_1)
    
    resultados = pd.concat([resultados, resultados_temp], ignore_index=True)
    # pls = ajustar_pls_letalidad(covid_municipal_0, caracteristicas_rf)
    # resultados = calificar_municipios_letalidad(resultados, pls, caracteristicas, etiqueta='i_PLS_RF_1')
    
    return resultados


In [ ]:
# exports

def calcular_periodo_vulnerabilidad(inicio, fin):
    inicio = pd.to_datetime(inicio, yearfirst=True)
    fin = pd.to_datetime(fin, yearfirst=True)
    
    fechas = pd.date_range(inicio, fin)
    resultados = []

    asegura_archivos_covid_disponibles(fechas)

    f = IntProgress(min=0, max=len(fechas) - 1) # instantiate the bar
    display(f) # display the bar
    
    for count, fecha in enumerate(fechas):
        covid_municipal = tabla_covid_indicadores_municipales(fecha.strftime("%y%m%d"))
        covid_municipal = agregar_tasas_municipales(covid_municipal)
     
        caracteristicas = caracteristicas_modelos_municipios(covid_municipal)
        pls = ajustar_pls_letalidad(covid_municipal, caracteristicas)
        df = calificar_municipios_letalidad_formato_largo(covid_municipal, pls, caracteristicas,
                                                    modelo='PLS', dia_ajuste=fecha.strftime("%y-%m-%d"))
        resultados.append(df)
        f.value = count
    
    resultados_df = pd.concat(resultados, ignore_index=True)
    
    return resultados_df


In [ ]:
serie_vulnerabilidad = calcular_periodo_vulnerabilidad('20-04-24', '20-06-07')

In [ ]:
serie_vulnerabilidad

,CLAVE_MUNICIPIO_RES,CLAVE_ENTIDAD_RES,MUNICIPIO_RES,geometry,RESULTADO,ENTIDAD_RES,conteo,defunciones,pt_2015,an_2015,...,total_de_camas_privados,total_camas,pob2020,CVE_MUN,covid_confrimados_100k,covid_defun_100k,tasa_covid_letal,modelo,dia_ajuste,valor
0,01001,01,AGUASCALIENTES,"POLYGON ((-102.09775 22.02325, -102.11598 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,126.0,2.0,877190,2.0570,...,43,1271,961977,001,13.098026,0.207905,1.587302,PLS,20-04-24,6.748512
1,01002,01,ASIENTOS,"POLYGON ((-102.04348 22.29392, -102.05861 22.3...",Positivo SARS-CoV-2,AGUASCALIENTES,1.0,0.0,46464,4.4207,...,0,0,50864,002,1.966027,0.000000,0.000000,PLS,20-04-24,10.554487
2,01003,01,CALVILLO,"POLYGON ((-102.57625 21.96778, -102.59749 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,0.0,0.0,56048,4.7805,...,0,46,60760,003,0.000000,0.000000,NaN,PLS,20-04-24,9.822440
3,01004,01,COSÍO,"POLYGON ((-102.26605 22.40372, -102.28993 22.4...",Positivo SARS-CoV-2,AGUASCALIENTES,0.0,0.0,15577,4.3233,...,0,0,16918,004,0.000000,0.000000,NaN,PLS,20-04-24,8.819903
4,01005,01,JESÚS MARÍA,"POLYGON ((-102.31034 22.03716, -102.33259 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,7.0,0.0,120405,3.2445,...,0,297,130184,005,5.377005,0.000000,0.000000,PLS,20-04-24,8.034288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110920,32054,32,VILLA HIDALGO,"POLYGON ((-101.65599 22.51381, -101.67475 22.5...",Positivo SARS-CoV-2,ZACATECAS,1.0,1.0,19155,8.1332,...,0,0,20177,054,4.956138,4.956138,100.000000,PLS,20-06-07,13.414194
110921,32055,32,VILLANUEVA,"POLYGON ((-102.69428 22.62230, -102.70306 22.6...",Positivo SARS-CoV-2,ZACATECAS,3.0,0.0,30240,4.7985,...,0,22,31804,055,9.432776,0.000000,0.000000,PLS,20-06-07,12.160276
110922,32056,32,ZACATECAS,"POLYGON ((-102.58542 22.81149, -102.59311 22.7...",Positivo SARS-CoV-2,ZACATECAS,74.0,5.0,146147,1.2432,...,0,337,155533,056,47.578327,3.214752,6.756757,PLS,20-06-07,9.348958
110923,32057,32,TRANCOSO,"POLYGON ((-102.24610 22.73451, -102.24578 22.7...",Positivo SARS-CoV-2,ZACATECAS,4.0,0.0,19413,5.6122,...,0,12,20285,057,19.719004,0.000000,0.000000,PLS,20-06-07,12.564649


In [ ]:
serie_vulnerabilidad[serie_vulnerabilidad.MUNICIPIO_RES == 'AGUASCALIENTES']

In [ ]:
#exports

def guardar_resultados(resultados):
    reporte_csv = resultados.rename(columns={'tasa_covid_conocidos': 'casos_100k',
                             'tasa_covid_defun': 'defunciones_100k',
                             'tasa_covid_letal': 'letalidad',
                             'i_PLS_0': 'i_vulnerabilidad'})
    
    resultados_dir = 'resultados'
    sem_ent_cols = ['CLAVE_ENTIDAD_RES', 'nom_ent'] + ['i_vuln_estatal', 'semaforo_estado', 'semaforo_tend_estado']
    sem_mun_cols = ['CLAVE_MUNICIPIO_RES', 'nom_ent', 'nom_mun'] + ['i_vulnerabilidad', 'semaforo_municipio']

    nombre_csv_mun = os.path.join(resultados_dir, 'i_semaforo_municipios.csv')
    nombre_csv_ent = os.path.join(resultados_dir, 'i_semaforo_entidades.csv')
    nombre_csv_full = os.path.join(resultados_dir, 'semaforo_municipios.csv')

    reporte[sem_mun_cols].to_csv(nombre_csv_mun, index=False)
    reporte[sem_ent_cols].drop_duplicates().to_csv(nombre_csv_ent, index=False)
    reporte.drop(columns='geometry').to_csv(nombre_csv_full, index=False)


In [ ]:
def mapas_serie_letalidad(serie_vulnerabilidad, gif=True):
    '''
        Recibe el DataFrame con la vulnerabilidad calculada para un rango de fechas y genera los mapas de cada fecha. 
    '''
    dir_salida = 'resultados/mapas/letalidad'
    if not os.path.exists(dir_salida):
        os.mkdir(dir_salida)
        
    serie_vulnerabilidad = serie_vulnerabilidad[['dia_ajuste', 'geometry', 'tasa_covid_letal', 'CLAVE_MUNICIPIO_RES']].copy()
    serie_vulnerabilidad = serie_vulnerabilidad.drop_duplicates()
    
    serie_vulnerabilidad.dia_ajuste = pd.to_datetime(serie_vulnerabilidad.dia_ajuste, yearfirst=True)
    inicio = serie_vulnerabilidad.dia_ajuste.min()
    fin = serie_vulnerabilidad.dia_ajuste.max()
    
    lista_mapas = []
    for fecha in pd.date_range(inicio, fin):
        datos_mapa = serie_vulnerabilidad[(serie_vulnerabilidad.dia_ajuste == fecha)]
        datos_mapa = datos_mapa[datos_mapa.tasa_covid_letal.notna()]
        fig = datos_mapa.plot(column='tasa_covid_letal', cmap='YlOrRd', scheme="equalinterval",
                        figsize=(10, 10), legend=True, k=10)
        # remove axis of chart
        fig.axis('off')

        # add a title
        fig.set_title('Letalidad COVID-19 Municipios de México', \
                  fontdict={'fontsize': '25', 'fontweight' : '3'})
        nombre_mapa = fecha.strftime("%y%m%d") + '_letalidad.jpg'
        ruta_archivo = os.path.join(dir_salida, nombre_mapa)
        lista_mapas.append(ruta_archivo)
        # position the annotation to the bottom left
        fig.annotate(fecha.strftime("%d/%m"),
                xy=(0.1, .225), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                fontsize=35)
        
        chart = fig.get_figure()
        chart.savefig(ruta_archivo, dpi=300)
        plt.close(chart)
    
    !convert -delay 60 -loop 0 {' '.join(lista_mapas)} 'letalidad_covid.gif'

    return lista_mapas
    


In [ ]:
lista_mapas = mapas_serie_letalidad(serie_vulnerabilidad)

In [ ]:
#exports

def mapas_serie_vulnerabilidad(serie_vulnerabilidad, modelo='PLS'):
    '''
        Recibe el DataFrame con la vulnerabilidad calculada para un rango de fechas y genera los mapas de cada fecha. 
    '''
    dir_salida = 'resultados/mapas'
    
    serie_vulnerabilidad = serie_vulnerabilidad.copy()    
    serie_vulnerabilidad.dia_ajuste = pd.to_datetime(serie_vulnerabilidad.dia_ajuste, yearfirst=True)
    inicio = serie_vulnerabilidad.dia_ajuste.min()
    fin = serie_vulnerabilidad.dia_ajuste.max()
    lista_mapas = []
    
    for fecha in pd.date_range(inicio, fin):
        datos_mapa = serie_vulnerabilidad[(serie_vulnerabilidad.dia_ajuste == fecha) &
                                         (serie_vulnerabilidad.modelo == modelo)]
        datos_mapa = datos_mapa[datos_mapa.valor.notna()]
        fig = datos_mapa.plot(column='valor', cmap='YlOrRd', scheme="Quantiles",
                        figsize=(10, 10), legend=True, k=5)
        # remove axis of chart
        fig.axis('off')

        # add a title
        fig.set_title('Vulnerabilidad COVID-19 Municipios de México', \
                  fontdict={'fontsize': '25', 'fontweight' : '3'})
        nombre_mapa = fecha.strftime("%y%m%d") + '_vulnerabilidad.jpg'
        ruta_archivo = os.path.join(dir_salida, nombre_mapa)
        lista_mapas.append(ruta_archivo)
        
        # position the annotation to the bottom left
        fig.annotate(fecha.strftime("%d/%m"),
                xy=(0.1, .225), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                fontsize=35)
        
        chart = fig.get_figure()
        chart.savefig(ruta_archivo, dpi=300)
        plt.close(chart)

    !convert -delay 60 -loop 0 {' '.join(lista_mapas)} 'vulnerabilidad_covid.gif'
    
    return lista_mapas

    
    

In [ ]:
mapas_serie_vulnerabilidad(serie_vulnerabilidad)

In [ ]:
from nbdev.export import notebook2script
notebook2script()